## **RagamBatik - Klasifikasi Gambar Batik**

# Import Library

In [ ]:
# Import library
import os
import random
import shutil
import datetime
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# Data Loading

In [ ]:
from google.colab import files

# Download Dataset
files.upload()

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d nadyams/indonesia-batik-dataset
!unzip -q indonesia-batik-dataset.zip -d indonesia-batik-dataset

# Data Preprocessing

Kelas `Madura_Gentongan` dihapus dari dataset karena hanya memiliki **8 gambar**, jumlah yang terlalu sedikit dibandingkan kelas lain. Hal ini dilakukan untuk mencegah ketidakseimbangan ekstrem yang dapat memengaruhi kinerja model.

In [ ]:
# Hapus kelas yang tidak diinginkan
original_dataset_dir = '/content/indonesia-batik-dataset'

folder_to_remove = os.path.join(original_dataset_dir, 'Madura_Gentongan')
if os.path.exists(folder_to_remove):
    shutil.rmtree(folder_to_remove)
    print("Kelas 'Madura_Gentongan' berhasil dihapus dari dataset.")
else:
    print("Folder 'Madura_Gentongan' tidak ditemukan.")

Kelas 'Madura_Gentongan' berhasil dihapus dari dataset.


In [ ]:
# Split dataset ke train/val/test
original_dataset_dir = '/content/indonesia-batik-dataset'
base_dir = '/content/batik-dataset-split'
split_ratio = (0.7, 0.15, 0.15)  # train, val, test

for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(base_dir, split), exist_ok=True)

classes = [cls for cls in os.listdir(original_dataset_dir) if os.path.isdir(os.path.join(original_dataset_dir, cls))]

for cls in tqdm(classes, desc="Membagi dataset"):
    src_path = os.path.join(original_dataset_dir, cls)
    files = os.listdir(src_path)
    random.shuffle(files)

    n_total = len(files)
    n_train = int(n_total * split_ratio[0])
    n_val = int(n_total * split_ratio[1])

    splits = {
        'train': files[:n_train],
        'val': files[n_train:n_train + n_val],
        'test': files[n_train + n_val:]
    }

    for split in ['train', 'val', 'test']:
        dst_dir = os.path.join(base_dir, split, cls)
        os.makedirs(dst_dir, exist_ok=True)
        for fname in splits[split]:
            shutil.copy(os.path.join(src_path, fname), os.path.join(dst_dir, fname))

print("Dataset berhasil dipisah ke dalam train/val/test.")

Membagi dataset: 100%|██████████| 23/23 [00:00<00:00, 30.15it/s]

Dataset berhasil dipisah ke dalam train/val/test.


In [ ]:
# Parameter
img_size = (224, 224)
batch_size = 32

# Augmentasi Gambar untuk train
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.15,
    shear_range=0.15,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

# Data Generators
train_generator = train_datagen.flow_from_directory(
    os.path.join(base_dir, 'train'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

val_generator = val_test_datagen.flow_from_directory(
    os.path.join(base_dir, 'val'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

test_generator = val_test_datagen.flow_from_directory(
    os.path.join(base_dir, 'test'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Hitung class_weight
labels = train_generator.classes
class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)
class_weight_dict = dict(enumerate(class_weights_array))

print("Class weights:", class_weight_dict)

Found 1532 images belonging to 23 classes.
Found 324 images belonging to 23 classes.
Found 340 images belonging to 23 classes.
Class weights: {0: np.float64(2.37888198757764), 1: np.float64(2.2202898550724637), 2: np.float64(2.37888198757764), 3: np.float64(0.7836317135549872), 4: np.float64(1.9031055900621119), 5: np.float64(0.2552057304680993), 6: np.float64(2.37888198757764), 7: np.float64(0.6343685300207039), 8: np.float64(2.37888198757764), 9: np.float64(2.466988727858293), 10: np.float64(2.37888198757764), 11: np.float64(2.37888198757764), 12: np.float64(2.37888198757764), 13: np.float64(2.2202898550724637), 14: np.float64(2.37888198757764), 15: np.float64(0.9515527950310559), 16: np.float64(0.59472049689441), 17: np.float64(0.4269788182831661), 18: np.float64(2.37888198757764), 19: np.float64(2.37888198757764), 20: np.float64(2.37888198757764), 21: np.float64(0.2632754768860629), 22: np.float64(1.1101449275362318)}


# Modelling

In [ ]:
# Build Model dengan Fine-tuning MobileNetV2
base_model = MobileNetV2(include_top=False, input_shape=img_size + (3,), weights='imagenet')
base_model.trainable = True  # unfreeze semua layer untuk fine-tuning

# Custom head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = Dropout(0.4)(x)
outputs = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=outputs)

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:
# Callbacks
checkpoint_path = 'best_batik_model.h5'
callbacks = [
    ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1),
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=4, verbose=1)
]

In [ ]:
# Training
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=70,
    callbacks=callbacks,
    class_weight=class_weight_dict
)

In [ ]:
# Evaluasi di test set
model.load_weights(checkpoint_path)
test_loss, test_acc = model.evaluate(test_generator)
print(f"Akurasi di test set: {test_acc:.4f}")

In [ ]:
# Load Model dari Checkpoint Terbaik
model.load_weights('best_batik_model.h5')
print("Model terbaik berhasil dimuat dari checkpoint.")

# Compile Ulang Model dengan Learning Rate Kecil
model.compile(
    optimizer=Adam(learning_rate=1e-5),  # lebih kecil agar fine-tuning halus
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
print("Model dikompilasi ulang dengan learning rate lebih kecil (1e-5).")

In [ ]:
# Fine-Tuning Lanjutan
fine_tune_epochs = 30
initial_epoch = 70

history_finetune = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=initial_epoch + fine_tune_epochs,
    initial_epoch=initial_epoch,
    callbacks=callbacks,
    class_weight=class_weight_dict
)

print("Fine-tuning lanjutan selesai.")

# Evaluation

In [ ]:
# Fungsi untuk Gabung Plot Training Awal + Fine-Tuning
def plot_training(history1, history2=None):
    acc = history1.history['accuracy']
    val_acc = history1.history['val_accuracy']
    loss = history1.history['loss']
    val_loss = history1.history['val_loss']

    if history2:
        acc += history2.history['accuracy']
        val_acc += history2.history['val_accuracy']
        loss += history2.history['loss']
        val_loss += history2.history['val_loss']

    epochs = range(1, len(acc) + 1)

    plt.figure(figsize=(14, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'b', label='Training acc')
    plt.plot(epochs, val_acc, 'r', label='Validation acc')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

plot_training(history, history_finetune)

In [ ]:
# Evaluasi di test set
test_loss, test_acc = model.evaluate(test_generator)
print(f"Akurasi di test set: {test_acc:.4f}")
print(f"Loss di test set: {test_loss:.4f}")

In [ ]:
# Confusion Matrix

# Prediksi kelas
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
y_true = test_generator.classes

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
labels = list(test_generator.class_indices.keys())

plt.figure(figsize=(14, 10))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot(xticks_rotation=90, cmap='Blues', values_format='d')
plt.title("Confusion Matrix - Test Set")
plt.show()

# Inference

In [ ]:
# Fungsi prediksi
def predict_image(img_path, model, class_indices, target_size=(224, 224)):
    # Load & preprocess image
    if not os.path.exists(img_path):
        print(f"Error: File not found at {img_path}")
        return None, None

    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Prediksi
    prediction = model.predict(img_array, verbose=0)
    predicted_class = np.argmax(prediction[0])
    class_labels = list(class_indices.keys())
    confidence = prediction[0][predicted_class]

    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Prediksi: {class_labels[predicted_class]} ({confidence:.2%})")
    plt.show()

    return class_labels[predicted_class], confidence

# Path Image
img_path = '/content/indonesia-batik-dataset/DKI_Jakarta_Ondel_Ondel/Ondel (26).jpg'

predict_image(img_path, model, test_generator.class_indices)

In [ ]:
# Mengambil prediksi dari model
Y_pred_all = model.predict(test_generator, verbose=1)
y_pred_all = np.argmax(Y_pred_all, axis=1)
confidences_all = np.max(Y_pred_all, axis=1)
y_true_all = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Mengumpulkan confidence untuk setiap kelas yang diprediksi dengan benar
average_confidences = {}

for i in range(len(y_true_all)):
    true_idx = y_true_all[i]
    pred_idx = y_pred_all[i]
    confidence = confidences_all[i]

    if true_idx == pred_idx:
        class_name = class_labels[true_idx]
        if class_name not in average_confidences:
            average_confidences[class_name] = []
        average_confidences[class_name].append(confidence)

# Menampilkan rata-rata confidence per kelas (untuk prediksi yang benar)
print("\nRata-rata Confidence Prediksi yang Benar per Kelas:")
for class_name in class_labels:
    confidences = average_confidences.get(class_name, [])
    if confidences:
        avg_conf = np.mean(confidences)
        print(f"- {class_name}: {avg_conf:.2%}")
    else:
        print(f"- {class_name}: (Tidak ada prediksi yang benar)")

# Rata-rata confidence keseluruhan (semua prediksi)
average_confidence_overall = np.mean(confidences_all)
print(f"\nRata-rata Confidence Keseluruhan (Semua Prediksi): {average_confidence_overall:.2%}")

# Rata-rata confidence hanya untuk prediksi yang benar
correct_indices = np.where(y_true_all == y_pred_all)[0]
if len(correct_indices) > 0:
    average_confidence_correct = np.mean(confidences_all[correct_indices])
    print(f"Rata-rata Confidence Hanya untuk Prediksi yang Benar: {average_confidence_correct:.2%}")
else:
    print("Tidak ada prediksi yang benar di test set.")

11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 141ms/step

Rata-rata Confidence Prediksi yang Benar per Kelas:
- Aceh_Pintu_Aceh: 99.91%
- Bali_Barong: 100.00%
- Bali_Merak: 90.61%
- Betawi_Tumpal: 100.00%
- DKI_Jakarta_Ondel_Ondel: 99.99%
- Jawa_Barat_Megamendung: 98.95%
- Jawa_Timur_Pring: 99.56%
- Kalimantan_Barat_Insang: 98.36%
- Kalimantan_Dayak: 100.00%
- Lampung_Gajah: 96.46%
- Maluku_Pala: 94.67%
- NTB_Lumbung: 99.97%
- Papua_Asmat: 99.99%
- Papua_Cendrawasih: 99.46%
- Papua_Tifa: 98.79%
- Solo_Parang: 92.10%
- Solo_Sidoluhur: 96.37%
- Solo_Truntum: 95.49%
- Sulawesi_Selatan_Lontara: 77.88%
- Sumatera_Barat_Rumah_Minang: 99.96%
- Sumatera_Utara_Boraspati: 93.15%
- Yogyakarta_Kawung: 97.22%
- Yogyakarta_Parang: 85.62%

Rata-rata Confidence Keseluruhan (Semua Prediksi): 94.25%
Rata-rata Confidence Hanya untuk Prediksi yang Benar: 96.80%


# Save Model

In [ ]:
!pip install tensorflowjs

In [ ]:
import tensorflowjs as tfjs

# Mengonversi model ke format TensorFlow.js
tfjs.converters.save_keras_model(model, 'model_tfjs')

failed to lookup keras version from the file,
    this is likely a weight only file


In [ ]:
import shutil
from google.colab import files

shutil.make_archive('model_tfjs', 'zip', 'model_tfjs')

files.download('model_tfjs.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>